#원본데이터 업로드

In [2]:
import pandas as pd
from matplotlib import pyplot as plt
plt.rc('font',family='HYGothic-Medium')   #그래프 한글 사용

In [ ]:
df = pd.read_csv("mlast.csv")  #UTF-8  CP949
df.head()

In [ ]:
print(df.info())
print(df.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3299 entries, 0 to 3298
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   영화제목    3299 non-null   object
 1   개봉일     3272 non-null   object
 2   매출액     3299 non-null   object
 3   매출액.1   3299 non-null   object
 4   누적매출액   3299 non-null   object
 5   관객수     3299 non-null   object
 6   누적관객수   3299 non-null   int64 
 7   스크린수    3299 non-null   object
 8   상영횟수    3299 non-null   int64 
 9   대표국적    3298 non-null   object
 10  국적      3298 non-null   object
 11  제작사     1149 non-null   object
 12  배급사     3263 non-null   object
 13  등급      3282 non-null   object
 14  장르      3293 non-null   object
 15  감독      3064 non-null   object
 16  주연      2953 non-null   object
 17  영화소개    3299 non-null   object
dtypes: int64(2), object(16)
memory usage: 464.0+ KB
None
(3299, 18)


#개봉일처리

nan 값 오늘날짜로 대체했습니다
개봉일이 2021-15-04로 12월 넘는 날이 있어 원래 개봉일 찾아서 넣어주었습니다.

In [ ]:
def date_change(g):
    if g == '2021-15-04':
        return pd.to_datetime('2020-09-03')
    if type(g) == type(' '):
        return pd.to_datetime(g)
    else:
        return pd.to_datetime('today')


In [ ]:
df['개봉날짜'] = df['개봉일'].apply(lambda x : date_change(x))

#영화소개글처리

##Okt 모델 적용

In [ ]:
#konlpy 설치
!pip install konlpy

In [ ]:
# Okt 토큰화
from konlpy.tag import Okt
okt = Okt()

##한글 불용어 사전 다운로드
로컬에서 사용시 첨부된 파일로 사용가능

In [ ]:
!wget https://raw.githubusercontent.com/byungjooyoo/Dataset/main/ko_stopwords.txt
with open("ko_stopwords.txt", 'r') as f:
    stopwords = []
    for line in f.readlines():
        stopwords.append(line.strip())
print('한글 불용어 개수:', len(stopwords))
# 불용어 수동 추가
more_stopwords = ['기자', '.', 'net', '<', '저작권자', '©', 'KOREA', '무단전재', '및', '>', '•', ',', '"']
for word in more_stopwords:
    stopwords.append(word)

한글 불용어 개수: 668


In [ ]:
#불용어 및 1자리수 글자 제거
def preprocessing(text,stopWord):
    stopPos = ['Suffix','Punctuation','Josa','Foreign','Alpha','Number','Verb','Adjective']
    word =[]
    for tag in text: #불용어 처리
        if tag[1] not in stopPos:
            if tag[0] not in stopWord:
                if len(tag[0]) != 1:  #1글자 제거
                    word.append(tag[0])
    return word

In [ ]:
df['body_pos'] = df['영화소개'].apply(lambda x : okt.pos(x, stem=True,norm=True) )

In [ ]:
df['body_key'] = df['body_pos'].apply(lambda x : preprocessing(x,stopwords))

##TFIDF

불용어 처리까지 끝낸 키값('body_key')사용

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['body_key'])
tfidf_matrix


<765x22194 sparse matrix of type '<class 'numpy.float64'>'
	with 181792 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cos_sim_df = pd.DataFrame(cosine_sim, index = df['00'] , columns=df['00'])
cos_sim_df

00,0,1,2,3,4,5,6,7,8,9,...,755,756,757,758,759,760,761,762,763,764
00,,,,,,,,,,,,,,,,,,,,,
0,1.000000,0.020341,0.063509,0.052485,0.027059,0.078742,0.008691,0.015010,0.065658,0.025638,...,0.005581,0.032475,0.027483,0.025388,0.000860,0.002314,0.027822,0.025594,0.013379,0.038713
1,0.020341,1.000000,0.085903,0.049400,0.064052,0.055550,0.017582,0.005026,0.042535,0.101014,...,0.012437,0.047088,0.022578,0.055597,0.001722,0.002252,0.053718,0.041028,0.002544,0.055663
2,0.063509,0.085903,1.000000,0.142432,0.097227,0.113684,0.023056,0.054883,0.168781,0.076519,...,0.009334,0.081139,0.066658,0.083987,0.004908,0.010676,0.065933,0.070474,0.002916,0.120953
3,0.052485,0.049400,0.142432,1.000000,0.072568,0.095748,0.012034,0.038057,0.121184,0.092696,...,0.008414,0.060724,0.059302,0.081904,0.001495,0.006768,0.054038,0.046853,0.004289,0.085950
4,0.027059,0.064052,0.097227,0.072568,1.000000,0.067868,0.014998,0.005399,0.058377,0.075822,...,0.016795,0.040368,0.033644,0.077162,0.001730,0.007838,0.059520,0.041798,0.004807,0.050441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760,0.002314,0.002252,0.010676,0.006768,0.007838,0.007099,0.001824,0.000000,0.007112,0.008141,...,0.001674,0.005958,0.001694,0.011654,0.000000,1.000000,0.006193,0.005011,0.000000,0.011221
761,0.027822,0.053718,0.065933,0.054038,0.059520,0.050953,0.025118,0.009120,0.055487,0.035906,...,0.013370,0.056449,0.033335,0.089501,0.006329,0.006193,1.000000,0.047368,0.007970,0.047769
762,0.025594,0.041028,0.070474,0.046853,0.041798,0.068098,0.016282,0.005619,0.036252,0.040496,...,0.007831,0.058725,0.024128,0.060909,0.006175,0.005011,0.047368,1.000000,0.016513,0.050099


In [ ]:
df['TFIDF'] = pd.DataFrame(tfidf_matrix)

In [ ]:
df.head(2)

,00,영화제목,매출액,매출액.1,누적매출액,관객수,누적관객수,스크린수,상영횟수,개봉날짜,...,주연_히메나 라마드리드,주연_히메쉬 파텔,주연_히스 레저,주연_히어로 파인즈 티핀,주연_히트맨,주연_힐러리 스웽크,주연_힐러리 클린턴,주연_힐미르 스나에르 구오나손,주연_힘의길,TFIDF
0,0,스파이더맨: 노 웨이 홈,"53,772,689,910",4.9%,"53,772,689,910","5,369,773",5369773,"2,948",137848,2021-12-15,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"(0, 10555)\t0.011590005470167282\n (0, 1476..."
1,1,남산의 부장들,"41,225,216,650",3.8%,"41,225,216,650","4,750,345",4750345,"1,659",140051,2020-01-22,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"(0, 20546)\t0.009349191719052916\n (0, 5965..."


In [ ]:
# @title


#국적,제작사, 배급사, 등급, 장르, 감독, 주연 인코딩

In [ ]:
def add_incoding(col_name):

    global df
    genre_dummies = df[col_name].str.get_dummies(',')

    genre_name = []
    for i in genre_dummies.columns:
        genre_name.append(col_name+'_'+i)

    genre_dummies.columns = genre_name
    df = pd.concat([df, genre_dummies], axis=1)

대표국적과 국적중 국적을 기준으로 인코딩

In [ ]:
add_incoding('국적')
add_incoding('제작사')
add_incoding('배급사')
add_incoding('등급')
add_incoding('장르')
add_incoding('감독')
add_incoding('주연')

#중복된 데이터 삭제

In [ ]:
df = df.drop(['개봉일','누적매출액','누적관객수','대표국적','국적','제작사','배급사','등급','장르','감독','주연','영화소개'],axis=1)

In [ ]:
print(df.shape)
df.info()

(3299, 20120)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3299 entries, 0 to 3298
Columns: 20120 entries, 영화제목 to 주연_힘의길
dtypes: datetime64[ns](1), int64(20111), object(8)
memory usage: 506.4+ MB


In [27]:
df.columns

Index(['id', '영화제목', '매출액', '매출액.1', '관객수', '스크린수', '상영횟수', '개봉날짜', 'body_pos',
       'body_key',
       ...
       '주연_히메나 라마드리드', '주연_히메쉬 파텔', '주연_히스 레저', '주연_히어로 파인즈 티핀', '주연_히트맨',
       '주연_힐러리 스웽크', '주연_힐러리 클린턴', '주연_힐미르 스나에르 구오나손', '주연_힘의길', 'TFIDF'],
      dtype='object', length=20120)

In [24]:
df.to_csv('3jo_prj_df1217.csv')

#모델적용위한 전처리

In [ ]:
X = df.copy()
y = df['누적관객수']
X.drop(['id','누적관객수','영화제목','개봉날짜','매출액','매출액.1','누적매출액','관객수','스크린수','상영횟수','body_key','body_pos', 'TFIDF'],inplace=True,axis=1)
X.shape, y.shape
X.info()

In [28]:
X.head(2)

,국적_그리스,국적_기타,국적_남아프리카공화국,국적_네덜란드,국적_노르웨이,국적_뉴질랜드,국적_대만,국적_덴마크,국적_독일,국적_러시아,...,주연_히로스에 료코,주연_히메나 라마드리드,주연_히메쉬 파텔,주연_히스 레저,주연_히어로 파인즈 티핀,주연_히트맨,주연_힐러리 스웽크,주연_힐러리 클린턴,주연_힐미르 스나에르 구오나손,주연_힘의길
0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
y.head(2)

0    5369773
1    4750345
Name: 누적관객수, dtype: int64

In [25]:
X.to_csv('3jo_X.csv')

In [26]:
y.to_csv('3jo_y.csv')